In [14]:
import seaborn as sns
import csv,json,os,statistics,time
import datetime as dt
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option('display.max_columns', 8)

In [15]:
df_sales=pd.read_csv('Original-Data/FACT_InternetSales.csv')
df_customers=pd.read_csv('Original-Data/DIM_Customers.csv')
df_products=pd.read_csv('Original-Data/DIM_Products.csv',keep_default_na=False)

# Data Preprocessing
## Handling duplicate value
### a. Products and Sales table
In Products table, many rows have the same ProductItemCode value but differ ProductKey and Product Status due to errors in collecting data.  
So we will use ProductItemCode column to detect duplicate and only keep the last occurrence of duplicated values <br>

In [16]:
print(df_products.duplicated(subset=['ProductItemCode'],keep='last').value_counts())
df_products[df_products.duplicated(subset=['ProductItemCode'],keep='last')]

False    504
True     102
dtype: int64


,ProductKey,ProductItemCode,Product Name,Sub Category,...,Product Line,Product Model Name,Product Description,Product Status
211,212,HL-U509-R,"Sport-100 Helmet, Red",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
212,213,HL-U509-R,"Sport-100 Helmet, Red",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
214,215,HL-U509,"Sport-100 Helmet, Black",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
215,216,HL-U509,"Sport-100 Helmet, Black",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
219,220,HL-U509-B,"Sport-100 Helmet, Blue",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
...,...,...,...,...,...,...,...,...,...
440,441,FR-R92B-48,"HL Road Frame - Black, 48",Road Frames,...,R,HL Road Frame,Our lightest and best quality aluminum frame m...,Outdate
442,443,FR-R92B-52,"HL Road Frame - Black, 52",Road Frames,...,R,HL Road Frame,Our lightest and best quality aluminum frame m...,Outdate
461,462,GL-H102-S,"Half-Finger Gloves, S",Gloves,...,S,Half-Finger Gloves,"Full padding, improved finger flex, durable pa...",Outdate
463,464,GL-H102-M,"Half-Finger Gloves, M",Gloves,...,S,Half-Finger Gloves,"Full padding, improved finger flex, durable pa...",Outdate


As you can see, there are 102 duplicated values based on ProductItemCode column. Most of them have the same ProductItemCode and Product Name but only differ in Product Status, so this leads to duplicated rows

In [17]:
df_products[df_products['ProductItemCode']=='HL-U509-R']

,ProductKey,ProductItemCode,Product Name,Sub Category,...,Product Line,Product Model Name,Product Description,Product Status
211,212,HL-U509-R,"Sport-100 Helmet, Red",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
212,213,HL-U509-R,"Sport-100 Helmet, Red",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Outdate
213,214,HL-U509-R,"Sport-100 Helmet, Red",Helmets,...,S,Sport-100,"Universal fit, well-vented, lightweight , snap...",Current


For example, Product with code HL-U509-R has 2 duplicated rows because of Product status. When deleteing, we only need to keep the last raw and remove the others <br>
But before deleting the duplicate values in Products table , we need to change ProductKey on df_sales because some of them are duplicated. We can do this by joining 2 tables and keep ProductItemCode for later use

In [18]:
# Joining 2 tables and keep ProductItemCode for df_sales
df_tempSales=pd.merge(df_products[['ProductKey','ProductItemCode']], df_sales, on='ProductKey', how='inner')

# Drop duplicates
df_products=df_products.drop_duplicates(subset=['ProductItemCode'],keep='last')


In [19]:
df_products

,ProductKey,ProductItemCode,Product Name,Sub Category,...,Product Line,Product Model Name,Product Description,Product Status
0,1,AR-5381,Adjustable Race,NULL,...,NULL,NULL,NULL,Current
1,2,BA-8327,Bearing Ball,NULL,...,NULL,NULL,NULL,Current
2,3,BE-2349,BB Ball Bearing,NULL,...,NULL,NULL,NULL,Current
3,4,BE-2908,Headset Ball Bearings,NULL,...,NULL,NULL,NULL,Current
4,5,BL-2036,Blade,NULL,...,NULL,NULL,NULL,Current
...,...,...,...,...,...,...,...,...,...
601,602,BB-8107,ML Bottom Bracket,Bottom Brackets,...,NULL,ML Bottom Bracket,Aluminum alloy cups; large diameter spindle.,Current
602,603,BB-9108,HL Bottom Bracket,Bottom Brackets,...,NULL,HL Bottom Bracket,Aluminum alloy cups and a hollow axle.,Current
603,604,BK-R19B-44,"Road-750 Black, 44",Road Bikes,...,R,Road-750,Entry level adult bike; offers a comfortable r...,Current
604,605,BK-R19B-48,"Road-750 Black, 48",Road Bikes,...,R,Road-750,Entry level adult bike; offers a comfortable r...,Current


In [20]:
df_tempSales['ProductKey'].nunique()

133

In [21]:
# Join 2 tables again
df_tempSales=pd.merge(df_products[['ProductKey','ProductItemCode']], df_tempSales, on='ProductItemCode', how='inner',suffixes=['_1','_2'])

# Replace the old duplicated Product key with new Product key (without duplicated)
df_tempSales['ProductKey']=df_tempSales['ProductKey_1']

# Drop excessed columns
df_tempSales=df_tempSales.drop(['ProductKey_1','ProductKey_2'],axis=1)
df_sales=df_tempSales.copy(deep=True)

In [22]:
df_sales.info()
df_sales['ProductKey'].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58168 entries, 0 to 58167
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ProductItemCode   58168 non-null  object 
 1   OrderDateKey      58168 non-null  int64  
 2   DueDateKey        58168 non-null  int64  
 3   ShipDateKey       58168 non-null  int64  
 4   CustomerKey       58168 non-null  int64  
 5   SalesOrderNumber  58168 non-null  object 
 6   SalesAmount       58168 non-null  float64
 7   ProductKey        58168 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 4.0+ MB


117

### b. Customer table

We determine duplicated customer based on 4 columns: First name, Last name, DateFirstPurchase, Customer CIty

In [23]:
df_customers.shape

(18484, 7)

In [24]:
df_customers=df_customers.drop_duplicates(subset=['First Name','Last Name','DateFirstPurchase','Customer City'],keep='last')
df_customers.shape

(18484, 7)

We can cleary see that customer table does not have duplicated value because row number is the same after deleted

### c. Import Product and Sales table to .csv after preprocessing and ready to use it for our Dashboard

In [26]:
df_products.to_csv('Products_edited.csv')
df_sales.to_csv('Sales_edited.csv')